 Please install Python 3.10+ in your VM Workstation

This notebook needs to run on the local machine so please download the notebooks and execute

Login to your azure account az login - device

In [ ]:
# Install all required packages
import sys
import subprocess

def install_package(package_name):
    try:
        __import__(package_name)
        print(f"✓ {package_name} is already installed")
    except ImportError:
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', package_name])
        print(f"✓ {package_name} installed successfully")

# Install essential packages
install_package('python-dotenv')
install_package('azure-identity')
install_package('azure-ai-ml')
install_package('tqdm')

# For safety, ensure azure-ai-ml is latest version
print("\nUpdating azure-ai-ml to latest version...")
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'azure-ai-ml', '--upgrade'])
print("✓ azure-ai-ml updated to latest version")

## Library Installation

First, let's install all the necessary libraries for model downloading and management. This includes:
- `python-dotenv` for environment variables management
- `azure-ai-ml` for Azure ML interactions
- `tqdm` for progress bars
- `azure-identity` for authentication

In [ ]:
! pip install azure-ai-ml -U  

In [ ]:
# Import required packages
import os
import time
import sys
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential
from tqdm.notebook import tqdm
from dotenv import load_dotenv

print("All packages successfully imported")

In [ ]:
# This cell is no longer needed as imports are handled in the previous cell
print("Import cell has been consolidated. You can delete this cell if desired.")

In [ ]:
# Get Azure ML credentials from environment variables
subscription_id = os.getenv('AZUREML_SUBSCRIPTION_ID')
resource_group = os.getenv('AZUREML_RESOURCE_GROUP')
workspace = os.getenv('AZUREML_WS_NAME')

# Print values for debugging (remove in production)
print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Workspace: {workspace}")

# Create ML client with the credentials
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

In [ ]:
ml_client.models.list()

## Download the Fine-Tuned Model

Now we'll download the fine-tuned Phi-4-Mini model from Azure ML Model Registry. This process will:

1. Connect to Azure ML using our credentials
2. Download the ONNX model (int4 quantized for CPU)
3. Display a progress bar during download
4. Show the download location and model size when complete

After downloading, you'll be able to use this model locally for inference.

In [ ]:
# Define model name and version
model_name = "fine-tuning-phi-4-mini-onnx-int4-cpu"
model_version = 1

print(f"\nStarting download of model: {model_name} (version {model_version})\n")

# Create a progress bar
with tqdm(total=100, desc="Downloading model", unit="%") as pbar:
    # Start a background thread to download the model
    download_future = ml_client.models.download_async(name=model_name, version=model_version)
    
    # Monitor the download progress
    while not download_future.done():
        time.sleep(0.5)
        # Update progress bar (approximate since we don't have actual progress info from the API)
        pbar.update(1)
        pbar.refresh()
        if pbar.n >= 95:  # Cap at 95% until confirmed complete
            pbar.n = 95
            pbar.refresh()

    # Get the result
    download_path = download_future.result()
    pbar.n = 100  # Set to 100% when complete
    pbar.refresh()

print(f"\nModel downloaded successfully to: {download_path}")

# Calculate and display the size of the downloaded model
total_size = 0
for path, dirs, files in os.walk(download_path):
    for f in files:
        fp = os.path.join(path, f)
        total_size += os.path.getsize(fp)

# Convert to appropriate size format (bytes, KB, MB, GB)
def format_size(size_bytes):
    if size_bytes < 1024:
        return f"{size_bytes} bytes"
    elif size_bytes < 1024**2:
        return f"{size_bytes/1024:.2f} KB"
    elif size_bytes < 102
for path, dirs, files in os.walk(download_path):
    for f in files:
        fp = os.path.join(path, f)
        total_size += os.path.getsize(fp)

# Convert to appropriate size format (bytes, KB, MB, GB)
def format_size(size_bytes):
    if size_bytes < 1024:
        return f"{size_bytes} bytes"
    elif size_bytes < 1024**2:
        return f"{size_bytes/1024:.2f} KB"
    elif size_bytes < 1024**3:
        return f"{size_bytes/(1024**2):.2f} MB"
    else:
        return f"{size_bytes/(1024**3):.2f} GB"

print(f"Total model size: {format_size(total_size)}")